## Dependencies and Setup

Load the target image, and test some triangles at different colour values

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using StaticArrays
using ImageFeatures
using IntervalSets
using Combinatorics
using StatsBase
using Zygote
using Symbolics

In [ ]:
target = float.(load("../lisa.png"))
hist = Serialization.deserialize("../output/simresult/simlog_100-prims_1000000-batch_10-epoch_100-refine.bin");

In [ ]:
tris = hist.history[end].shapes;

In [ ]:
function drawstate(cols, bg, alpha)
    initial = zeros(RGB{Float32}, size(target)) .+ bg
    for k in eachindex(tris)
        draw!(initial, tris[k], RGBA{Float32}(cols[k].r, cols[k].g, cols[k].b, alpha), RasterAlgorithmScanline())
    end
    initial
end

In [ ]:
simalpha = 0.75

In [ ]:
@time alphacols, alphabg = alpharecolor(target, tris, simalpha, RasterAlgorithmScanline());
alphacols

In [ ]:
@time opaquecols, opaquebg = opaquerecolor(target, tris, RasterAlgorithmScanline());
opaquecols

In [ ]:
plots = []
for alphaval in [simalpha, 1.0]
    for idx in 1:2
        cols, bg = [(alphacols, alphabg), (opaquecols, opaquebg)][idx]
        name = ["Alpha-$simalpha Recolor", "Opaque Recolor"][idx]
        img = drawstate(cols, bg, alphaval)
        loss = imloss(img, target, AELoss())
        plt = plot(img, size=size(target), showaxis=false, ticks=false, title="$name \n α=$alphaval, loss=$loss", titlefontsize=12)
        push!(plots, plt)
    end
end

In [ ]:
plot(plots..., size=size(target) .* 3)